In [72]:
from utils.make_sample import make_sample
import Levenshtein
import pandas as pd
import numpy as np

In [62]:
path_to_file = "./data/redir"

In [106]:
data = make_sample(path_to_file)

In [107]:
data.shape

(3003, 4)

In [112]:
def preprocess_query_category(data):
    data['query'] = data['query'].apply(preprocessing_text)
    data['category_name'] = data['category_name'].apply(preprocessing_text)

    return data

In [116]:
X = preprocess_query_category(data.drop(columns=['is_redirect']))
y = data['is_redirect']

In [120]:
X

,query,category_id,category_name
0,dior хайлайтер,3327621,хайлайтеры
1,dior хайлатер,3327621,хайлайтеры
2,diorskin mineral nude luminizer,3327621,хайлайтеры
3,хайдайтер dior,3327621,хайлайтеры
4,хайлайтер dior,3327621,хайлайтеры
...,...,...,...
2998,парфюмерная вода yves saint laurent black opiu...,873904,нишевая парфюмерия
2999,крем гель для умывания dove увлажняющий,874090,патчи
3000,жидкие патчи для глаз levrana forever young,873248,сыворотки
3001,кисть для пудры sisley kabuki brush,873874,для глаз


In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [142]:
vectorizer = TfidfVectorizer(analyzer="char_wb",
                             ngram_range=(2, 3),
                             max_features=1000)

In [ ]:
vectorizer.fit_transform(all_text)

In [102]:
def make_features(data):
    """
    Функция для создания фичей.
    """
    
    # TODO переписать это гомно в один цикл.
    
    data['len_of_query'] = data['query'].apply(lambda text: len(text))
    data['num_of_word_in_query'] = data['query'].apply(lambda text: len(text.split()))
    
    data['category_name'] = data['category_name'].apply(preprocessing_text)
    data['len_of_category_name'] = data['category_name'].apply(lambda text: len(text))
    data['num_of_word_in_category_name'] = data['category_name'].apply(lambda text: len(text.split()))
    
    data['is_query_contains_brand'] = data['query'].apply(is_query_contains_brand)
    
# data.drop(columns=['category_id', 'query', 'category_name'], inplace=True)
    
    return data

In [104]:
def preprocessing_text(s):
    # Функция для предобработки текста запроса/названия категории
    #:param s: str
    # замена некорректного символа
    s = s.replace("&#039;", "'")
    # удаление знаков препинания  
    s = s.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'))
    # приведение к нижнему регистру
    s = s.lower()
    # удаление лишних пробелов
    s = ' '.join(s.split())
    return s


def is_query_contains_brand(query):
    brands = pd.read_csv(path_to_file + "/unique_brands.csv")
    brands = [str(brand) for brand in brands.iloc[:, 0]]
    for brand in brands:
        res = query.find(brand)
        if res != -1:
            return 1
    return 0


def get_levenshtein_distance_between(query_text, category_name):
    return Levenshtein.distance(query_text, category_name)